# Project Description
In this project, I will load csv file to sql database and make a connection to that database through pandas and make a query to extract some data. For this project I have used **PopSQL** editor. This project is just for my learning purpose. 

# About Dataset
"The name of the dataset is 'Online Retail'. This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."
Source of the dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/00352/

# Needed Libraries

In [1]:
import pandas as pd
import mysql.connector
import re

In [2]:
df = pd.read_csv("/Users/shreesapkota/Documents/Integrify/SymLinkHDD/APPLE_HDD/Data/Online Retail.csv",delimiter=';',)

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1.12.2010 8:26,"2,55",17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1.12.2010 8:26,"3,39",17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1.12.2010 8:26,"2,75",17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1.12.2010 8:26,"3,39",17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1.12.2010 8:26,"3,39",17850.0,United Kingdom


# Data Cleaning and Processing
I need to do a little bit of cleaning the data and change the datatypes which I will do as follows. 

SQL primary key index starts from 1. That is why I am changing the index here. 

In [4]:
df.index = df.index+1

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 1 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  object 
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


As we can see that CustomerID has many null values, so I will remove all the rows where CustomerID is null. 

In [6]:
# Dropping all null values
df.dropna(inplace=True)

It is necessary to change the datatype of some columns. Hence, I will change them as follows. 

In [7]:
# To datetime datatype
df.InvoiceDate = pd.to_datetime(df.InvoiceDate,)

If we check the UnitPrice column the strings are like "2,15" form. So, to convert this to float, I will change ',' to '.' and covert the datatype to float. 

In [8]:
df.UnitPrice=[re.sub(',','.',string=i) for i in df.UnitPrice.values]

In [9]:
df.UnitPrice = df.UnitPrice.astype(float)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 1 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


Now, lets save the file into csv format so that we can load this file to sql server. 

In [11]:
#df.to_csv('Online_Retail.csv')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 1 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


# Create table first
In the PopSQL, I will create a table first with all the columns names. I will also create an extra column to make it a primary key as I didn't find other columns fit for this role.

Before loading the data into the database, I created table as below.

CREATE TABLE Online_Retail (

    Invoice_id INT AUTO_INCREMENT,     
    InvoiceNo VARCHAR(15,
    StockCode VARCHAR(6),   
    Description VARCHAR(150),
    Quantity INT,
    InvoiceDate DATETIME,
    UnitPrice FLOAT,
    CustomerID FLOAT,
    Country VARCHAR(22),
    PRIMARY KEY(Invoice_id)
);

# Load csv file into the database
I have the csv file saved in the local machine(saved in utf-8 separated by comma).I will load this file to the database. I have already created a database named 'my_database'. I used the following commands in the command line. 

#mysql -u root -p (start mysql with root user and password)
#use my_database (name of the database)
#show tables (shows all the tables that are created)

**Loading the csv file into the table***

#LOAD DATA LOCAL INFILE "/Users/shreesapkota/Documents/Integrify/SymLinkHDD/APPLE_HDD/Data/Online Retail12.csv" 

#INTO TABLE Online_Retail FIELDS TERMINATED BY ';' 

#LINES TERMINATED by '\n' 

#IGNORE 1 LINES (Index_num, InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country);

**Error: "Loading local data is disabled; this must be enabled on both the client and server sides"**
#set global local_infile=true

#LOAD DATA LOCAL INFILE "/Users/shreesapkota/Documents/Integrify/SymLinkHDD/APPLE_HDD/Data/Online Retail12.csv" INTO TABLE Online_Retail 
FIELDS TERMINATED BY ';' 
LINES TERMINATED by '\n' 
IGNORE 1 LINES (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country);

Screenshot from commandline shown below when loading the data. 

<img src="code_screenshot.png"/>

Image from PopSQL editor shown below. 

<img src="a.png" width=600 height=600 />

# Connecting to sql server through python
In order to make a connection to sql server, we need to install mysql-connector-python and we will do it as follows. 

In [ ]:
#!pip install mysql-connector-python

We wil now create a connection to our mysql server in order to connect to our database as shown as below. In the parameter, "passwd=pw", I have already defined my password in the variable name 'pw' 

In [19]:
connection = mysql.connector.connect(host='localhost', user='root', passwd=pw, database='my_database')

Now that we have created a connection to our database, we can use pandas to make sql query to our database as follows. Our table has more than 400 thousand rows in the database but we will send a query request for only 100 rows of data. 

# Querying first 100 rows

In [20]:
Only_100 = pd.read_sql_query(con = connection, sql="SELECT * FROM Online_Retail LIMIT 100",index_col="Invoice_id")

In [21]:
Only_100.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
Invoice_id,,,,,,,,
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom
2,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
3,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
5,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom


# Conclusion
This was my first attempt to create a database, load data to database and connect the database through python. It was successfull. 